In [1]:
import pybullet as p
import time
import cv2
import pybullet_data
import numpy as np

In [2]:
physicsClient = p.connect(p.DIRECT)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-10)
# planeId = p.loadURDF("plane.urdf")
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])

In [3]:
mass = -1
visual_tray = p.createVisualShape(p.GEOM_BOX,
                                    halfExtents=[0.2, 0.27, 0.005],
                                    rgbaColor=[0,1,1,1])

shape_tray = p.createCollisionShape(p.GEOM_BOX,
                                  halfExtents=[0.2, 0.27, 0.005])

basePosition = [0, 0.47, 0.005]
baseOrientation = [0,0,0,1]

trayId = p.createMultiBody(mass,
                              shape_tray,
                              visual_tray,
                              basePosition,
                              baseOrientation)
box_1_r = 0.03
shape_box_1 = p.createCollisionShape(p.GEOM_BOX,
                                  halfExtents=[box_1_r, box_1_r, box_1_r])

box_2_r = 0.05
shape_box_2 = p.createCollisionShape(p.GEOM_BOX,
                                  halfExtents=[box_2_r, box_2_r, 0.7])

basePosition = [0, 0.50, 0.06]
baseOrientation = [0, 0, 0, 1]

visual_box_1 = p.createVisualShape(p.GEOM_BOX,
                                    halfExtents=[box_1_r, box_1_r, box_1_r],
                                    rgbaColor=[1,0,1,1])


box1 = p.createMultiBody(mass, shape_box_1, visual_box_1, basePosition,
                              baseOrientation)
basePosition = [0.05, 0.3, 0.06]

visual_box_2 = p.createVisualShape(p.GEOM_BOX,
                                    halfExtents=[box_2_r, box_2_r, 0.7],
                                    rgbaColor=[1,1,0,1])


box2= p.createMultiBody(mass, shape_box_2, visual_box_2, basePosition,
                              baseOrientation)


In [4]:
width, height = 108, 108

viewMatrix = p.computeViewMatrix([0, 0.47, 1], [0, 0.47, -1],
                                      [0, 1, 0])

projMatrix = p.computeProjectionMatrixFOV(
    fov=30, aspect=1, nearVal=0.1, farVal=1)


light = {
    "diffuse": 0.4,
    "ambient": 0.5,
    "spec": 0.2,
    "dir": [10, 10, 100],
    "col": [1, 1, 1]}

img = p.getCameraImage(
    width=width,
    height=height,
    projectionMatrix=projMatrix,
    viewMatrix=viewMatrix,
    shadow=1,
    lightAmbientCoeff=light["ambient"],
    lightDiffuseCoeff=light["diffuse"],
    lightSpecularCoeff=light["spec"],
    lightDirection=light["dir"],
    lightColor=light["col"]
)

In [5]:
rgb = np.array(img[2], dtype=np.float).reshape(height, width, 4) / 255
rgb[:, :, 3], rgb[:, :, 2] = rgb[:, :, 2], rgb[:, :, 0]
rgb[:, :, 0] = rgb[:, :, 3]

# rgb = rgb[:, 11:-11, :]
pixel_ratio = 2
width, height = 54 * pixel_ratio, 54 * pixel_ratio

width_clip = int(14 * (pixel_ratio / 2))

rgb_map = rgb[:, :, 0:3]
rgb_map = rgb_map[:, width_clip:-width_clip, :]

depth_map = np.array(img[3], dtype=np.float).reshape(height, width)
depth_map = depth_map[:, 14:-14]

rgb_d = np.dstack((rgb_map, depth_map))

In [6]:
depth_map = 1 - 0.1/(1-(1-0.1)*depth_map)

In [10]:
depth_map = depth_map * 100


In [19]:
depth_map.shape

(108, 80)

In [49]:
depth_map = list(map(lambda x: np.round(x), depth_map))

# round(depth_map)
depth_map


[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [69]:
s = np.dstack((r,g,b,depth))
s

array([[[1.        , 1.        , 1.        , 0.80168867],
        [1.        , 1.        , 1.        , 0.78951961],
        [1.        , 1.        , 1.        , 0.77789366],
        [1.        , 1.        , 1.        , 0.77413321],
        [1.        , 1.        , 1.        , 0.77037275],
        [1.        , 1.        , 1.        , 0.77358675],
        [1.        , 1.        , 1.        , 0.78343189],
        [1.        , 1.        , 1.        , 0.79327703]],

       [[1.        , 1.        , 1.        , 0.80168867],
        [1.        , 1.        , 1.        , 0.78951961],
        [1.        , 1.        , 1.        , 0.77735054],
        [1.        , 1.        , 1.        , 0.76518148],
        [1.        , 1.        , 1.        , 0.75301242],
        [1.        , 1.        , 1.        , 0.76285756],
        [1.        , 1.        , 1.        , 0.77270263],
        [1.        , 1.        , 1.        , 0.78254777]],

       [[1.        , 1.        , 1.        , 0.80168867],
        [1

In [55]:
s = np.append(rgb, depth_map, axis=2)

s
# s.shape

array([[[1.        , 1.        , 1.        , 1.        , 0.80168867],
        [1.        , 1.        , 1.        , 1.        , 0.78951961],
        [1.        , 1.        , 1.        , 1.        , 0.77789366],
        [1.        , 1.        , 1.        , 1.        , 0.77413321],
        [1.        , 1.        , 1.        , 1.        , 0.77037275],
        [1.        , 1.        , 1.        , 1.        , 0.77358675],
        [1.        , 1.        , 1.        , 1.        , 0.78343189],
        [1.        , 1.        , 1.        , 1.        , 0.79327703]],

       [[1.        , 1.        , 1.        , 1.        , 0.80168867],
        [1.        , 1.        , 1.        , 1.        , 0.78951961],
        [1.        , 1.        , 1.        , 1.        , 0.77735054],
        [1.        , 1.        , 1.        , 1.        , 0.76518148],
        [1.        , 1.        , 1.        , 1.        , 0.75301242],
        [1.        , 1.        , 1.        , 1.        , 0.76285756],
        [1.       

In [32]:
s

array([[[1.        , 1.        , 1.        , 1.        , 0.80168867],
        [1.        , 1.        , 1.        , 1.        , 0.78951961],
        [1.        , 1.        , 1.        , 1.        , 0.77789366],
        [1.        , 1.        , 1.        , 1.        , 0.77413321],
        [1.        , 1.        , 1.        , 1.        , 0.77037275],
        [1.        , 1.        , 1.        , 1.        , 0.77358675],
        [1.        , 1.        , 1.        , 1.        , 0.78343189],
        [1.        , 1.        , 1.        , 1.        , 0.79327703]],

       [[1.        , 1.        , 1.        , 1.        , 0.80168867],
        [1.        , 1.        , 1.        , 1.        , 0.78951961],
        [1.        , 1.        , 1.        , 1.        , 0.77735054],
        [1.        , 1.        , 1.        , 1.        , 0.76518148],
        [1.        , 1.        , 1.        , 1.        , 0.75301242],
        [1.        , 1.        , 1.        , 1.        , 0.76285756],
        [1.       

In [ ]:
for i in range (10000):
    p.stepSimulation()
    time.sleep(1./240.)
    # render()

cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)

p.disconnect()

In [78]:
a = []
a

if not a:
    print("chi")

chi


In [1]:
a = []
b= 1
c=2
a.append(b,c)
a

TypeError: append() takes exactly one argument (2 given)